# Observables

Observables provide a unified way to access a large quantity of figures resulting from various
computations on lattices. They may be used in parameter scans, matching, response matrices…

Observables values may be scalars or numpy arrays of any shape.

AT provides a number of specific observables sharing a common interface, inherited from the
{py:class}`.Observable` base class. They are:
- {py:class}`.RingObservable`: Any user-defined property depending upon *ring*
- {py:class}`.ElementObservable`: Any user-defined property at a specific location along the lattice
- {py:class}`.OrbitObservable`: {math}`x_{co}`…,
- {py:obj}`.GlobalOpticsObservable`: tunes, damping times…,
- {py:class}`.LocalOpticsObservable`: {math}`\beta`, {math}`\eta`…,
- {py:class}`.MatrixObservable`: {math}`T_{ij}`…,
- {py:class}`.TrajectoryObservable`: {math}`x, p_x`…,
- {py:class}`.EmittanceObservable`: {math}`\epsilon_x`…,
- {py:class}`.LatticeObservable`: attributes of lattice elements,
- {py:class}`.GeometryObservable`

An Observable has optional {py:attr}`~.Observable.target`, {py:attr}`~.Observable.weight` and {py:attr}`~.Observable.bounds` attributes for matching. After evaluation, it has the following main properties:
- {py:attr}`~.Observable.value`
- {py:attr}`~.Observable.weighted_value`: `value / weight`
- {py:attr}`~.Observable.deviation`:  `value - target`
- {py:attr}`~.Observable.weighted_deviation`:  `(value - target)/weight`
- {py:attr}`~.Observable.residual`:  `((value - target)/weight)**2`

Custom Observables may be created by providing the adequate evaluation function.

For evaluation, observables must be grouped in an {py:class}`.ObservableList` which optimises the computation, avoiding redundant function calls. {py:class}`.ObservableList` provides the {py:meth}`~.ObservableList.evaluate` method, and the
{py:attr}`~.ObservableList.values`, {py:attr}`~.ObservableList.deviations`,
{py:attr}`~.ObservableList.residuals` and {py:attr}`~.ObservableList.sum_residuals` properties, among others.

This example shows how to declare various Observables, how to evaluate them and how to extract and display their values.

## Setup the environment

In [1]:
import at
import numpy as np
from importlib.resources import files, as_file

In [2]:
from at import (
    ObservableList,
    RingObservable,
    OrbitObservable,
    GlobalOpticsObservable,
    LocalOpticsObservable,
    MatrixObservable,
    TrajectoryObservable,
    EmittanceObservable,
    LatticeObservable,
    GeometryObservable,
)

## Load a test lattice

In [3]:
fname = "hmba.mat"
with as_file(files("machine_data") / fname) as path:
    hmba_lattice = at.load_lattice(path)

## Create Observables

Create an empty ObservableList:

In [4]:
obs1 = ObservableList()

Horizontal closed orbit on all Monitors:

In [5]:
obs1.append(OrbitObservable(at.Monitor, axis="x"))

Create a 2{sup}`nd` ObservableList:

In [6]:
obs2 = ObservableList()

Vertical $\beta$ at all monitors, with a target and bounds.

The vertical $\beta$ is constrained in the interval
[*target*+*low_bound* *target*+*up_bound*], so here [*-Infinity 7.0*]

The residual will be zero within the interval.

In [7]:
obs2.append(
    LocalOpticsObservable(
        at.Monitor, "beta", plane=1, target=7.0, bounds=(-np.inf, 0.0)
    )
)

check the concatenation of ObservableLists:

In [8]:
allobs = obs1 + obs2

Full transfer matrix to `BPM02`:

In [9]:
allobs.append(MatrixObservable("BPM_02"))

Maximum of vertical beta on monitors:

In [10]:
allobs.append(LocalOpticsObservable(at.Monitor, "beta", plane="v", statfun=np.amax))

First 4 coordinates of the closed orbit at Quadrupoles:

In [11]:
allobs.append(
    LocalOpticsObservable(
        at.Quadrupole, "closed_orbit", plane=slice(4), target=0.0, weight=1.0e-6
    )
)

Position along the lattice of all quadrupoles:

In [12]:
allobs.append(LocalOpticsObservable(at.Quadrupole, "s_pos"))

Phase advance between elements 33 and 101 in all planes:

First, let's define a custom evaluation function:

In [13]:
def phase_advance(elemdata):
    mu = elemdata.mu
    return mu[-1] - mu[0]

Then create the Observable. The evaluation function should return one value per refpoint (2 here). Alternatively,
it may return a single value (the difference, here), but then one must set `summary=True`. `all_points` is set to `True` to force the evaluation of the phase advance at all points, to avoid phase jumps.

In [14]:
allobs.append(
    LocalOpticsObservable([33, 101], phase_advance, all_points=True, summary=True)
)

Horizontal tune with the integer part:

In [15]:
allobs.append(GlobalOpticsObservable("tune", plane=0, use_integer=True))

Total phase advance at the end of the lattice (all planes):

In [16]:
allobs.append(LocalOpticsObservable(at.End, "mu"))

Horizontal W-function:

In [17]:
allobs.append(LocalOpticsObservable(at.Sextupole, "W", plane='x'))

Chromaticity in all planes:

In [18]:
allobs.append(GlobalOpticsObservable("chromaticity"))

Average of sextupole strengths:

In [19]:
allobs.append(LatticeObservable(at.Sextupole, "H", statfun=np.mean))

Strengths of all sextupoles:

In [20]:
allobs.append(LatticeObservable(at.Sextupole, "PolynomB", index=2))

Horizontal emittance:

In [21]:
allobs.append(EmittanceObservable("emittances", plane="x"))

Ring circumference:

In [22]:
def circumference(ring):
    return ring.get_s_pos(len(ring))[0]

allobs.append(RingObservable(circumference))

p{sub}`x` component of the trajectory on all monitors:

In [23]:
allobs.append(TrajectoryObservable(at.Monitor, axis="px"))

In [24]:
allobs.append(GeometryObservable(at.Monitor, "x"))

## Evaluation

An input trajectory is required for the trajectory Observable

In [25]:
r_in = np.zeros(6)
r_in[0] = 0.001
r_in[2] = 0.001
values = allobs.evaluate(hmba_lattice.enable_6d(copy=True), r_in=r_in, dp=0.0, initial=True)

### Extract a single Observable value
(phase advance between elements 3 and 101):

In [26]:
allobs[6].value

array([ 9.38969042e+00,  2.99742405e+00, -4.29831951e-15])

### Get the list of all Observable values:

In [27]:
allobs.values

[array([-3.02197154e-09,  4.50706100e-07,  4.08215752e-07,  2.37905632e-08,
        -1.31787026e-08,  2.47236653e-08, -2.95318224e-08, -4.05608197e-07,
        -4.47409217e-07, -2.24856457e-09]),
 array([5.30279703, 7.17604152, 6.55087808, 2.31448878, 3.40498444,
        3.405044  , 2.3146451 , 6.55106241, 7.17614175, 5.30283837]),
 array([[[-1.08194106e+00,  3.18809568e+00,  0.00000000e+00,
           0.00000000e+00,  8.22407787e-02, -1.72158067e-05],
         [-6.80522735e-01,  1.08099571e+00,  0.00000000e+00,
           0.00000000e+00,  4.90131193e-02, -1.02601216e-05],
         [ 0.00000000e+00,  0.00000000e+00,  7.55929650e-01,
           3.87059271e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -6.79279293e-01,
          -2.15524755e+00,  0.00000000e+00,  0.00000000e+00],
         [-1.13309291e-08, -1.08618451e-07,  0.00000000e+00,
           0.00000000e+00,  9.99995907e-01, -2.09333332e-04],
         [ 2.93742438e-03,  6.73567963e-02,  0.0000

### Get a pretty output of all Observables.

As no variation was made, *Actual* values are always equal to *Initial* values.

The deviation is zero for all Observables for which no *target* was specified

In [28]:
print(allobs)


    location              Initial            Actual         Low bound        High bound         deviation 
orbit[x]
    BPM_01           -3.02197e-09      -3.02197e-09               -                 -                 0.0 
    BPM_02            4.50706e-07       4.50706e-07               -                 -                 0.0 
    BPM_03            4.08216e-07       4.08216e-07               -                 -                 0.0 
    BPM_04            2.37906e-08       2.37906e-08               -                 -                 0.0 
    BPM_05           -1.31787e-08      -1.31787e-08               -                 -                 0.0 
    BPM_06            2.47237e-08       2.47237e-08               -                 -                 0.0 
    BPM_07           -2.95318e-08      -2.95318e-08               -                 -                 0.0 
    BPM_08           -4.05608e-07      -4.05608e-07               -                 -                 0.0 
    BPM_09           -4.474